In [1]:
import pandas as pd
import datetime
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Datos del ayuntamiento

In [2]:
datos_camaras = pd.read_csv('PBD2/datos_modelo_aforo.csv', sep=';').drop(['Unnamed: 0'], axis=1)

In [3]:
datos_camaras.head()

,FECHA,HORA,IDENTIFICADOR,PEATONES,NÚMERO_DISTRITO,DISTRITO,NOMBRE_VIAL,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION,LATITUD,LONGITUD,Festivo,dia_semana,mes
0,2021-01-01 0:00,0:00,PERM_PEA02_PM01,110.0,1,Centro,Calle Fuencarral,22,28004,Calle peatonal,40.422009,-3.700892,True,Friday,January
1,2021-01-01 0:00,0:00,PERM_PEA03_PM01,111.0,1,Centro,Calle San Bernardo,36,28015,Acera Pares,40.423803,-3.707379,True,Friday,January
2,2021-01-01 0:00,0:00,PERM_PEA04_PM01,15.0,1,Centro,Calle Hortaleza,18,28004,Acera Pares,40.421072,-3.700490,True,Friday,January
3,2021-01-01 0:00,0:00,PERM_PEA05_PM01,44.0,1,Centro,Carrera de San Jerónimo,6,28014,Acera Pares,40.416641,-3.701247,True,Friday,January
4,2021-01-01 0:00,0:00,PERM_PEA06_PM01,49.0,1,Centro,Calle Atocha,95,28012,Acera Impares,40.411109,-3.696471,True,Friday,January


In [4]:
datos_camaras['FECHA']=datos_camaras['FECHA'].astype(str)
datos_camaras["FECHA"]

0          2021-01-01 0:00
1          2021-01-01 0:00
2          2021-01-01 0:00
3          2021-01-01 0:00
4          2021-01-01 0:00
                ...       
301763    2019-12-31 23:00
301764    2019-12-31 23:00
301765    2019-12-31 23:00
301766    2019-12-31 23:00
301767    2019-12-31 23:00
Name: FECHA, Length: 301768, dtype: object

In [5]:
datos_camaras['FECHA'] = datos_camaras['FECHA'].str.replace("-","/")
datos_camaras['FECHA']= datos_camaras['FECHA'].str.slice(0, 10)
datos_camaras["FECHA"]

0         2021/01/01
1         2021/01/01
2         2021/01/01
3         2021/01/01
4         2021/01/01
             ...    
301763    2019/12/31
301764    2019/12/31
301765    2019/12/31
301766    2019/12/31
301767    2019/12/31
Name: FECHA, Length: 301768, dtype: object

In [6]:
datos_camaras['HORA']=datos_camaras['HORA'].astype(str)
datos_camaras['FECHA']=datos_camaras['FECHA']+' '+datos_camaras['HORA']

In [7]:
datos_camaras['FECHA']

0          2021/01/01 0:00
1          2021/01/01 0:00
2          2021/01/01 0:00
3          2021/01/01 0:00
4          2021/01/01 0:00
                ...       
301763    2019/12/31 23:00
301764    2019/12/31 23:00
301765    2019/12/31 23:00
301766    2019/12/31 23:00
301767    2019/12/31 23:00
Name: FECHA, Length: 301768, dtype: object

In [8]:
datos_camaras['FECHA'] = datos_camaras['FECHA'].str.replace("/","-")

In [9]:
datos_camaras.head()

,FECHA,HORA,IDENTIFICADOR,PEATONES,NÚMERO_DISTRITO,DISTRITO,NOMBRE_VIAL,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION,LATITUD,LONGITUD,Festivo,dia_semana,mes
0,2021-01-01 0:00,0:00,PERM_PEA02_PM01,110.0,1,Centro,Calle Fuencarral,22,28004,Calle peatonal,40.422009,-3.700892,True,Friday,January
1,2021-01-01 0:00,0:00,PERM_PEA03_PM01,111.0,1,Centro,Calle San Bernardo,36,28015,Acera Pares,40.423803,-3.707379,True,Friday,January
2,2021-01-01 0:00,0:00,PERM_PEA04_PM01,15.0,1,Centro,Calle Hortaleza,18,28004,Acera Pares,40.421072,-3.700490,True,Friday,January
3,2021-01-01 0:00,0:00,PERM_PEA05_PM01,44.0,1,Centro,Carrera de San Jerónimo,6,28014,Acera Pares,40.416641,-3.701247,True,Friday,January
4,2021-01-01 0:00,0:00,PERM_PEA06_PM01,49.0,1,Centro,Calle Atocha,95,28012,Acera Impares,40.411109,-3.696471,True,Friday,January


## Añadimos los datos de 2022

In [10]:
datos_2022 = pd.read_csv('PBD2/peatones_2022.csv', sep=';').drop(['Unnamed: 0'], axis=1)

### Completamos los datos

In [11]:
datos_2022 = datos_2022.reindex(columns = datos_2022.columns.tolist() + ["LONGITUD","LATITUD",'NÚMERO_DISTRITO','NOMBRE_VIAL','DISTRITO','NÚMERO','CÓDIGO_POSTAL','OBSERVACIONES_DIRECCION'])

In [12]:
datos_2022.columns

Index(['FECHA', 'PEATONES', 'IDENTIFICADOR', 'LONGITUD', 'LATITUD',
       'NÚMERO_DISTRITO', 'NOMBRE_VIAL', 'DISTRITO', 'NÚMERO', 'CÓDIGO_POSTAL',
       'OBSERVACIONES_DIRECCION'],
      dtype='object')

In [13]:
longitudes=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).LONGITUD.unique()
for n in array_unicos:
    longitudes.append(n[0])

latitudes=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).LATITUD.unique()
for n in array_unicos:
    latitudes.append(n[0])

numeros_distrito=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).NÚMERO_DISTRITO.unique()
for n in array_unicos:
    numeros_distrito.append(n[0])
    
distritos=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).DISTRITO.unique()
for n in array_unicos:
    distritos.append(n[0])

nombres=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).NOMBRE_VIAL.unique()
for n in array_unicos:
    nombres.append(n[0])
    
numeros=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).NÚMERO.unique()
for n in array_unicos:
    numeros.append(n[0])
     
codigos=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).CÓDIGO_POSTAL.unique()
for n in array_unicos:
    codigos.append(n[0])
    
observaciones=[]
array_unicos=datos_camaras.groupby(['IDENTIFICADOR']).OBSERVACIONES_DIRECCION.unique()
for n in array_unicos:
    observaciones.append(n[0])
    

identificadores=datos_camaras.IDENTIFICADOR.unique()

In [14]:
todas_longitudes=[]
for n in longitudes:
    todas_longitudes=todas_longitudes+[n] * 8760

todas_latitudes=[]
for n in latitudes:
    todas_latitudes=todas_latitudes+[n] * 8760

todos_distritos=[]
for n in distritos:
    todos_distritos=todos_distritos+[n] * 8760

todos_numeros_distrito=[]
for n in numeros_distrito:
    todos_numeros_distrito=todos_numeros_distrito+[n] * 8760

todos_nombres=[]
for n in nombres:
    todos_nombres=todos_nombres+[n] * 8760

todos_numeros=[]
for n in numeros:
    todos_numeros=todos_numeros+[n] * 8760

todos_codigos=[]
for n in codigos:
    todos_codigos=todos_codigos+[n] * 8760

todos_observaciones=[]
for n in observaciones:
    todos_observaciones=todos_observaciones+[n] * 8760


In [15]:
datos_2022['LONGITUD']=todas_longitudes
datos_2022['LATITUD']=todas_latitudes
datos_2022['NÚMERO_DISTRITO']=todos_numeros_distrito
datos_2022['NOMBRE_VIAL']=todos_nombres
datos_2022['DISTRITO']=todos_distritos
datos_2022['NÚMERO']=todos_numeros
datos_2022['CÓDIGO_POSTAL']=todos_codigos
datos_2022['OBSERVACIONES_DIRECCION']=todos_observaciones

In [16]:
datos_2022['FECHA']= datos_2022['FECHA'].str.slice(0, 16)

In [17]:
datos_2022

,FECHA,PEATONES,IDENTIFICADOR,LONGITUD,LATITUD,NÚMERO_DISTRITO,NOMBRE_VIAL,DISTRITO,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION
0,2022-01-01 00:00,0.000000,PERM_PEA02_PM01,-3.700892,40.422009,1,Calle Fuencarral,Centro,22,28004,Calle peatonal
1,2022-01-01 01:00,0.000000,PERM_PEA02_PM01,-3.700892,40.422009,1,Calle Fuencarral,Centro,22,28004,Calle peatonal
2,2022-01-01 02:00,0.000000,PERM_PEA02_PM01,-3.700892,40.422009,1,Calle Fuencarral,Centro,22,28004,Calle peatonal
3,2022-01-01 03:00,0.000000,PERM_PEA02_PM01,-3.700892,40.422009,1,Calle Fuencarral,Centro,22,28004,Calle peatonal
4,2022-01-01 04:00,0.000000,PERM_PEA02_PM01,-3.700892,40.422009,1,Calle Fuencarral,Centro,22,28004,Calle peatonal
...,...,...,...,...,...,...,...,...,...,...,...
166435,2022-12-31 19:00,404.785221,PERM_PEA19_PM01,-3.713569,40.415111,1,Calle Bailén,Centro,10,28013,Acera Pares
166436,2022-12-31 20:00,430.001947,PERM_PEA19_PM01,-3.713569,40.415111,1,Calle Bailén,Centro,10,28013,Acera Pares
166437,2022-12-31 21:00,408.474188,PERM_PEA19_PM01,-3.713569,40.415111,1,Calle Bailén,Centro,10,28013,Acera Pares
166438,2022-12-31 22:00,352.524057,PERM_PEA19_PM01,-3.713569,40.415111,1,Calle Bailén,Centro,10,28013,Acera Pares


In [18]:
datos_2022.to_csv("PBD2/PEATONES_2022_COMPLETO.csv",sep=";")

In [19]:
datos_camaras2=datos_camaras.iloc[:, range(12)].drop(['HORA'], axis=1)
datos_camaras2

,FECHA,IDENTIFICADOR,PEATONES,NÚMERO_DISTRITO,DISTRITO,NOMBRE_VIAL,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION,LATITUD,LONGITUD
0,2021-01-01 0:00,PERM_PEA02_PM01,110.0,1,Centro,Calle Fuencarral,22,28004,Calle peatonal,40.422009,-3.700892
1,2021-01-01 0:00,PERM_PEA03_PM01,111.0,1,Centro,Calle San Bernardo,36,28015,Acera Pares,40.423803,-3.707379
2,2021-01-01 0:00,PERM_PEA04_PM01,15.0,1,Centro,Calle Hortaleza,18,28004,Acera Pares,40.421072,-3.700490
3,2021-01-01 0:00,PERM_PEA05_PM01,44.0,1,Centro,Carrera de San Jerónimo,6,28014,Acera Pares,40.416641,-3.701247
4,2021-01-01 0:00,PERM_PEA06_PM01,49.0,1,Centro,Calle Atocha,95,28012,Acera Impares,40.411109,-3.696471
...,...,...,...,...,...,...,...,...,...,...,...
301763,2019-12-31 23:00,PERM_PEA14_PM01,1026.0,7,Chamberí,Alberto Aguilera,56,28015,Acera Pares,40.430469,-3.713298
301764,2019-12-31 23:00,PERM_PEA15_PM01,87.0,1,Centro,Calle Toledo,23,28005,Acera Impares,40.412891,-3.707464
301765,2019-12-31 23:00,PERM_PEA17_PM01,249.0,2,Arganzuela,Ronda de Valencia,16,28012,Acera Pares,40.405367,-3.701798
301766,2019-12-31 23:00,PERM_PEA18_PM01,487.0,1,Centro,Calle Alcalá,34,28014,Acera Pares,40.418256,-3.697732


In [20]:
datos_finales= pd.concat([datos_camaras2, datos_2022])

In [21]:
datos_finales

,FECHA,IDENTIFICADOR,PEATONES,NÚMERO_DISTRITO,DISTRITO,NOMBRE_VIAL,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION,LATITUD,LONGITUD
0,2021-01-01 0:00,PERM_PEA02_PM01,110.000000,1,Centro,Calle Fuencarral,22,28004,Calle peatonal,40.422009,-3.700892
1,2021-01-01 0:00,PERM_PEA03_PM01,111.000000,1,Centro,Calle San Bernardo,36,28015,Acera Pares,40.423803,-3.707379
2,2021-01-01 0:00,PERM_PEA04_PM01,15.000000,1,Centro,Calle Hortaleza,18,28004,Acera Pares,40.421072,-3.700490
3,2021-01-01 0:00,PERM_PEA05_PM01,44.000000,1,Centro,Carrera de San Jerónimo,6,28014,Acera Pares,40.416641,-3.701247
4,2021-01-01 0:00,PERM_PEA06_PM01,49.000000,1,Centro,Calle Atocha,95,28012,Acera Impares,40.411109,-3.696471
...,...,...,...,...,...,...,...,...,...,...,...
166435,2022-12-31 19:00,PERM_PEA19_PM01,404.785221,1,Centro,Calle Bailén,10,28013,Acera Pares,40.415111,-3.713569
166436,2022-12-31 20:00,PERM_PEA19_PM01,430.001947,1,Centro,Calle Bailén,10,28013,Acera Pares,40.415111,-3.713569
166437,2022-12-31 21:00,PERM_PEA19_PM01,408.474188,1,Centro,Calle Bailén,10,28013,Acera Pares,40.415111,-3.713569
166438,2022-12-31 22:00,PERM_PEA19_PM01,352.524057,1,Centro,Calle Bailén,10,28013,Acera Pares,40.415111,-3.713569


In [22]:
datos_finales.to_csv("PBD2/PEATONES_TOTALES.csv",sep=";")

In [23]:
datos_carto=datos_finales
datos_carto['FECHA']=datos_carto['FECHA'].str.slice(0, 10)

In [24]:
datos_carto.to_csv("PBD2/PEATONES_TOTALES_CARTO.csv",sep=";")